# Manage Bedrock Knowledge Bases with Boto3

# Knowledge Bases for Amazon Bedrock PoC

This notebook provides sample code for building an **empty OpenSearch Serverless (OSS) index, Amazon Bedrock knowledge base and ingest documents into the index**. IllumiDesk plans on creating some resources with the IaaC repo, specifically, set up IAM Roles, IAM Policies (with the appropriate permissions) and attach said roles to the service(s) that need to interact with AWS Bedrock.

  A data pipeline that ingests documents (typically stored in Amazon S3) into a knowledge base i.e. a vector database such as Amazon OpenSearch Service Serverless (AOSS) so that it is available for lookup when a question is received.

#### Key Findings and Notes

1. By default, AWS Bedrock uses one S3 bucket per data source and a specific data source is attached to an AWS Bedrok Knowledge Base ID. Since AWS restricts the amount of S3 buckets by region, the workaround is to use the concept of `S3 prefixes` to identify the `Campus` and `Context Label` associated with an AWS Bedrock Knowledge Base.
2. An OpenSearch Serverless Collection has to be created _before_ creating an AWS Bedrock Knowledge Base.
3. An S3 bucket has to be available _before_ an AWS Bedrock Knowledge Base ID is associated to a OpenSearch Collection ARN and Data Source (S3) bucket.
4. The data pipeline is included with the AWS Bedrock Knowledge Base service. This abstracts away the need to manage the pipeline independently, such as with LangChain, etc.
5. The data pipeline uses AWS's Titan Embed model by default. As of this writing we can't change it.
6. The metadata is included when retrieving context from the Bedrock Knowledge Base OASS collection.
7. If we implement the AWS Bedrock Knowledge Base feature, then we can remove the dedicated OpenSearch Domain which requires dedicated instances.
8. Retrieval from the collection index has two options: the `RetreiveAndGenerate API` and the `Retrieve API`.
   - The `RetreiveAndGenerate API` provided by Knowledge Bases for Amazon Bedrock converts user queries into embeddings, searches the knowledge base, get the relevant results, augment the prompt and then invoking a LLM to generate the response.
   - The `Retreive API` provided by Knowledge Bases for Amazon Bedrock converts user queries into embeddings, searches the knowledge base, and returns the relevant results, which allows the user to customize how to augment the prompt before sending the request to the LLM generative AI model. The output of the `Retrieve API` includes the the `retrieved text chunks`, the `location type` and `URI` of the source data, as well as the relevance `scores` of the retrievals.
9. Logging is an important component for any LLM framework. Most examples with AWS Bedrock store logs in an S3 bucket and/or CloudWatch. Logging is not covered in detail with this notebook.

> The differences between the `RetrieveAndGenerate API` and the `Retrieve API` are also known as `Fully Managed RAG` and `Customized RAG Workflow`, respectively.

#### Steps:

- Create Amazon Bedrock Knowledge Base execution role with necessary policies for accessing data from S3 and writing embeddings into OSS.
- Create a Vector Store with [OpenSearch Serverless](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-setup-oss.html). The vector store in the context of AWS Bedrock Knowledge Bases are known as `Collections`.
- Create an S3 bucket which will work as the source for your data ingestion pipeline.
- [Add files to S3 bucket for data ingestion](!https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-setup.html). The S3 bucket in the context of the AWS Bedrock Knowledge Base is a `Data Source`.

- Create an empty OpenSearch serverless index.
- Download documents
- Create Amazon Bedrock knowledge base
- Create a data source within knowledge base which will connect to Amazon S3
- Start an ingestion job using KB APIs which will read data from s3, chunk it, convert chunks into embeddings using Amazon Titan Embeddings model and then store these embeddings in AOSS. All of this without having to build, deploy and manage the data pipeline.
- Retrieve context from the Bedrock Knowledge Base to agument the Generative AI's model response.


#### Illustrations

##### Data Ingestion

![data_ingestion.png](./images/data_ingestion.png)

[Source](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/knowledge-bases/0_create_ingest_documents_test_kb.ipynb)

##### Retrieval and Generate (`RetrieveAndGenerate API`)

![retrieveAndGenerate.png](./images/retrieveAndGenerate.png)

[Source](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/knowledge-bases/1_managed-rag-kb-retrieve-generate-api.ipynb)

##### Retrieval (`Retrieve API`)

![retrieveAPI.png](./images/retrieveAPI.png)

[Source](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/knowledge-bases/2_customized-rag-retreive-api-claude-v2.ipynb)

Refer to the Example Notebooks [located here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/knowledge-bases) for a step by step walk through using the `boto3` (Python SDK) client.

Refer to the following flow chart to view the process flow. (Original located [here](https://lucid.app/lucidchart/1603d006-3e59-408a-aec8-1773fb59abcd/edit?viewport_loc=588%2C5239%2C4983%2C2586%2ChcvUVn2nC3dG&invitationId=inv_1f7c1bcd-e0db-4ee1-8406-cd6758d97137)).

![bedrock_flow.png](./images/bedrock_flow.png)
